# Library import

In [2]:
import os
from typing import Tuple, Any, Callable, List, Optional, Union

import cv2
import timm
import torch
import numpy as np
import pandas as pd
import albumentations as A
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.model_selection import StratifiedKFold

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset Class

In [3]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        root_dir: str, 
        info_df: pd.DataFrame, 
        transform: Callable,
        is_inference: bool = False
    ):
        # 데이터셋의 기본 경로, 이미지 변환 방법, 이미지 경로 및 레이블을 초기화합니다.
        self.root_dir = root_dir  # 이미지 파일들이 저장된 기본 디렉토리
        self.transform = transform  # 이미지에 적용될 변환 처리
        self.is_inference = is_inference # 추론인지 확인
        self.image_paths = info_df['image_path'].tolist()  # 이미지 파일 경로 목록
        
        if not self.is_inference:
            self.targets = info_df['target'].tolist()  # 각 이미지에 대한 레이블 목록

    def __len__(self) -> int:
        # 데이터셋의 총 이미지 수를 반환합니다.
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        # 주어진 인덱스에 해당하는 이미지를 로드하고 변환을 적용한 후, 이미지와 레이블을 반환합니다.
        img_path = os.path.join(self.root_dir, self.image_paths[index])  # 이미지 경로 조합
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 이미지를 BGR 컬러 포맷의 numpy array로 읽어옵니다.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR 포맷을 RGB 포맷으로 변환합니다.
        image = self.transform(image)  # 설정된 이미지 변환을 적용합니다.

        if self.is_inference:
            return image
        else:
            target = self.targets[index]  # 해당 이미지의 레이블
            return image, target  # 변환된 이미지와 레이블을 튜플 형태로 반환합니다. 

# Transform Class

In [4]:
class UnsharpMask(A.ImageOnlyTransform):
    def __init__(self, kernel_size=5, sigma=1.0, amount=1.0, threshold=0, always_apply=False, p=1.0):
        super(UnsharpMask, self).__init__(always_apply, p)
        self.kernel_size = kernel_size
        self.sigma = sigma
        self.amount = amount
        self.threshold = threshold

    def apply(self, image, **params):
        return self.unsharp_mask(image)

    def unsharp_mask(self, image):
        blurred = cv2.GaussianBlur(image, (self.kernel_size, self.kernel_size), self.sigma)
        sharpened = cv2.addWeighted(image, 1.0 + self.amount, blurred, -self.amount, 0)
        if self.threshold > 0:
            low_contrast_mask = np.absolute(image - blurred) < self.threshold
            np.copyto(sharpened, image, where=low_contrast_mask)
        return sharpened

class AlbumentationsTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 그레이스케일 변환, 언샤프 마스크, 정규화, 텐서 변환
        common_transforms = [
            A.Resize(224, 224),  # 이미지를 224x224 크기로 리사이즈
            A.ToGray(p=1.0),  # 그레이스케일 변환
            UnsharpMask(kernel_size=5, sigma=1.0, amount=1.0, threshold=10, p=1.0),  # 언샤프 마스크 적용
            A.Normalize(mean=[0.5], std=[0.5]),  # 그레이스케일 이미지에 맞는 정규화
            ToTensorV2(),  # albumentations에서 제공하는 PyTorch 텐서 변환
        ]
        
        if is_train:
            # 훈련용 변환: 랜덤 크롭, 랜덤 수평 뒤집기, 랜덤 회전 추가
            self.transform = A.Compose(
                [   
                    A.HorizontalFlip(p=0.9),  # 90% 확률로 이미지를 수평 뒤집기
                    A.Rotate(limit=30),  # 최대 30도 회전
                    A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=0.9), # 20% 어둡게
                    A.GaussianBlur(blur_limit=(3, 5), p=0.6), # 약간의 블러 추가
                    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5), # 50% 확률로 블럭 추가
                    A.GridDistortion(always_apply=False, p=1, num_steps=1, distort_limit=(-0.03, 0.05), interpolation=2, border_mode=0, value=(0, 0, 0), mask_value=None)
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = A.Compose(common_transforms)

    def __call__(self, image) -> torch.Tensor:
        # 이미지가 NumPy 배열인지 확인
        if not isinstance(image, np.ndarray):
            raise TypeError("Image should be a NumPy array (OpenCV format).")
        
        # 이미지에 변환 적용 및 결과 반환
        transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용
        
        return transformed['image']  # 변환된 이미지의 텐서를 반환

# class AlbumentationsTransform:
#     def __init__(self, is_train: bool = True):
#         # 공통 변환 설정: 이미지 리사이즈, 정규화, 텐서 변환
#         common_transforms = [
#             A.Resize(224, 224),  # 이미지를 224x224 크기로 리사이즈
#             A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
#             ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
#         ]
        
#         if is_train:
#             # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 랜덤 밝기 및 대비 조정 추가
#             self.transform = A.Compose(
#                 [
#                     A.HorizontalFlip(p=0.9),  # 90% 확률로 이미지를 수평 뒤집기
#                     A.Rotate(limit=30),  # 최대 30도 회전
#                     A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=0.9), # 20% 어둡게
#                     A.GaussianBlur(blur_limit=(3, 5), p=0.6), # 약간의 블러 추가
#                     A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5), # 50% 확률로 블럭 추가
#                     A.GridDistortion(always_apply=False, p=1, num_steps=1, distort_limit=(-0.03, 0.05), interpolation=2, border_mode=0, value=(0, 0, 0), mask_value=None)
#                 ] + common_transforms
#             )
#         else:
#             # 검증/테스트용 변환: 공통 변환만 적용
#             self.transform = A.Compose(common_transforms)

#     def __call__(self, image) -> torch.Tensor:
#         # 이미지가 NumPy 배열인지 확인
#         if not isinstance(image, np.ndarray):
#             raise TypeError("Image should be a NumPy array (OpenCV format).")
        
#         # 이미지에 변환 적용 및 결과 반환
#         transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용
        
#         return transformed['image']  # 변환된 이미지의 텐서를 반환

In [5]:
class TransformSelector:
    """
    이미지 변환 라이브러리를 선택하기 위한 클래스.
    """
    def __init__(self, transform_type: str):

        # 지원하는 변환 라이브러리인지 확인
        if transform_type in ["torchvision", "albumentations"]:
            self.transform_type = transform_type
        
        else:
            raise ValueError("Unknown transformation library specified.")

    def get_transform(self, is_train: bool):
        
        # 선택된 라이브러리에 따라 적절한 변환 객체를 생성
        if self.transform_type == 'albumentations':
            transform = AlbumentationsTransform(is_train=is_train)
        
        return transform

# Model Class

In [6]:
class TimmModel(nn.Module):
    """
    Timm 라이브러리를 사용하여 다양한 사전 훈련된 모델을 제공하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TimmModel, self).__init__()
        self.model = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=num_classes
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [7]:
class TorchvisionModel(nn.Module):
    """
    Torchvision에서 제공하는 사전 훈련된 모델을 사용하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TorchvisionModel, self).__init__()
        self.model = models.__dict__[model_name](pretrained=pretrained)
        
        # 모델의 최종 분류기 부분을 사용자 정의 클래스 수에 맞게 조정
        if 'fc' in dir(self.model):
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Linear(num_ftrs, num_classes)
        
        elif 'classifier' in dir(self.model):
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [8]:
class ModelSelector:
    """
    사용할 모델 유형을 선택하는 클래스.
    """
    def __init__(
        self, 
        model_type: str, 
        num_classes: int, 
        **kwargs
    ):
        
        # 모델 유형에 따라 적절한 모델 객체를 생성
        if model_type == 'torchvision':
            self.model = TorchvisionModel(num_classes=num_classes, **kwargs)
        elif model_type == 'timm':
            self.model = TimmModel(num_classes=num_classes, **kwargs)
        else:
            raise ValueError("Unknown model type specified.")

    def get_model(self) -> nn.Module:

        # 생성된 모델 객체 반환
        return self.model

# Loss Class

In [9]:
class FocalLoss(nn.Module):
    """
    Focal Loss implementation for multi-class classification.
    """
    def __init__(self, alpha, gamma, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, outputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # cross entropy loss
        ce_loss = F.cross_entropy(outputs, targets, reduction='none')
        
        # 각 클래스 softmax
        probs = torch.exp(-ce_loss)
        
        # focal loss 식
        focal_loss = self.alpha * (1 - probs) ** self.gamma * ce_loss
        
        # return mean OR sum
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [10]:
class Loss(nn.Module):
    """
    모델의 손실함수를 계산하는 클래스.
    """
    def __init__(self):
        super(Loss, self).__init__()
        # self.loss_fn = nn.CrossEntropyLoss()
        self.loss_fn = FocalLoss(alpha=1, gamma=2)

    def forward(
        self, 
        outputs: torch.Tensor, 
        targets: torch.Tensor
    ) -> torch.Tensor:
    
        return self.loss_fn(outputs, targets)

# Trainer Class

In [11]:
class Trainer:
    def __init__(
        self, 
        model: nn.Module, 
        device: torch.device, 
        train_loader: DataLoader, 
        val_loader: DataLoader, 
        optimizer: optim.Optimizer,
        scheduler: optim.lr_scheduler,
        loss_fn: torch.nn.modules.loss._Loss, 
        epochs: int,
        result_path: str,
        fold: int,
    ):
        # 클래스 초기화: 모델, 디바이스, 데이터 로더 등 설정
        self.model = model  # 훈련할 모델
        self.device = device  # 연산을 수행할 디바이스 (CPU or GPU)
        self.train_loader = train_loader  # 훈련 데이터 로더
        self.val_loader = val_loader  # 검증 데이터 로더
        self.optimizer = optimizer  # 최적화 알고리즘
        self.scheduler = scheduler # 학습률 스케줄러
        self.loss_fn = loss_fn  # 손실 함수
        self.epochs = epochs  # 총 훈련 에폭 수
        self.fold = fold # fold 추가
        self.result_path = result_path  # 모델 저장 경로
        self.best_models = [] # 가장 좋은 상위 3개 모델의 정보를 저장할 리스트
        self.lowest_loss = float('inf') # 가장 낮은 Loss를 저장할 변수

    def save_model(self, epoch, loss):
        # 모델 저장 경로 설정
        os.makedirs(self.result_path, exist_ok=True)

        # 현재 에폭 모델 저장
        current_model_path = os.path.join(self.result_path, f'model_fold_{self.fold}_epoch_{epoch}_loss_{loss:.4f}.pt')
        torch.save(self.model.state_dict(), current_model_path)

        # 최상위 3개 모델 관리
        self.best_models.append((loss, epoch, current_model_path))
        self.best_models.sort()
        if len(self.best_models) > 3:
            _, _, path_to_remove = self.best_models.pop(-1)  # 가장 높은 손실 모델 삭제
            if os.path.exists(path_to_remove):
                os.remove(path_to_remove)

        # 가장 낮은 손실의 모델 저장
        if loss < self.lowest_loss:
            self.lowest_loss = loss
            best_model_path = os.path.join(self.result_path, 'best_model.pt')
            torch.save(self.model.state_dict(), best_model_path)
            print(f"Save {epoch}epoch result. Loss = {loss:.4f}")

    def train_epoch(self) -> float:
        # 한 에폭 동안의 훈련을 진행
        self.model.train()
        
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        progress_bar = tqdm(self.train_loader, desc="Training", leave=False)
        
        for images, targets in progress_bar:
            images, targets = images.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.loss_fn(outputs, targets)
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            total_loss += loss.item()
          
            _, preds = torch.max(outputs, 1)  # 예측된 클래스
            correct_predictions += (preds == targets).sum().item()  # 맞춘 예측 개수
            total_samples += targets.size(0)  # 전체 샘플 수
            
            progress_bar.set_postfix(loss=loss.item())
        
        accuracy = correct_predictions / total_samples * 100  # 정확도 계산
        print(f"Train Accuracy: {accuracy:.2f}%")
            
        
        return total_loss / len(self.train_loader)

    def validate(self) -> float:
        # 모델의 검증을 진행
        self.model.eval()
        
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        progress_bar = tqdm(self.val_loader, desc="Validating", leave=False)
        
        with torch.no_grad():
            for images, targets in progress_bar:
                images, targets = images.to(self.device), targets.to(self.device)
                outputs = self.model(images)    
                loss = self.loss_fn(outputs, targets)
                total_loss += loss.item()
                
                # 정확도 계산
                _, preds = torch.max(outputs, 1)  # 예측된 클래스
                correct_predictions += (preds == targets).sum().item()  # 맞춘 예측 개수
                total_samples += targets.size(0)  # 전체 샘플 수

                progress_bar.set_postfix(loss=loss.item())
  
        accuracy = correct_predictions / total_samples * 100  # 정확도 계산
        print(f"Validation Accuracy: {accuracy:.2f}%")
        
        return total_loss / len(self.val_loader)

    def train(self) -> None:
        # 전체 훈련 과정을 관리
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")
            
            train_loss = self.train_epoch()
            val_loss = self.validate()
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}\n")

            self.save_model(epoch, val_loss)
            self.scheduler.step()

# Model Training

In [12]:
# 학습에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [13]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정.
traindata_dir = "../data/train"
traindata_info_file = "../data/train.csv"
save_result_path = "../train_result"

In [14]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
train_info = pd.read_csv(traindata_info_file)

# 총 class의 수를 측정.
num_classes = len(train_info['target'].unique())

# 각 class별로 8:2의 비율이 되도록 학습과 검증 데이터를 분리.
train_df, val_df = train_test_split(
    train_info, 
    test_size=0.2,
    stratify=train_info['target']
)



In [15]:
# 학습에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
train_transform = transform_selector.get_transform(is_train=True)
val_transform = transform_selector.get_transform(is_train=False)

# 학습에 사용할 Dataset을 선언.
train_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform
)
val_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=val_df,
    transform=val_transform
)

# 학습에 사용할 DataLoader를 선언.
train_loader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=64, 
    shuffle=False
)

In [16]:
# 학습에 사용할 Model을 선언.
model_selector = ModelSelector(
    model_type='timm', 
    num_classes=num_classes,
    model_name='resnet101', 
    pretrained=True
)
model = model_selector.get_model()

# 선언된 모델을 학습에 사용할 장비로 셋팅.
model.to(device)

TimmModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, mome

In [17]:

# 학습에 사용할 optimizer를 선언하고, learning rate를 지정
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.001
)

In [18]:
# 스케줄러 초기화
scheduler_step_size = 30  # 매 30step마다 학습률 감소
scheduler_gamma = 0.1  # 학습률을 현재의 10%로 감소

# 한 epoch당 step 수 계산
steps_per_epoch = len(train_loader)

# 2 epoch마다 학습률을 감소시키는 스케줄러 선언
epochs_per_lr_decay = 10
scheduler_step_size = steps_per_epoch * epochs_per_lr_decay

scheduler = optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=scheduler_step_size, 
    gamma=scheduler_gamma
)

In [19]:
# StratifiedKFold 설정 (10-fold 교차 검증)
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 'train_info'에서 레이블 리스트 추출
labels = train_info['target'].tolist()

models = []

# 교차 검증 폴드별 데이터 분리 및 학습 루프
for fold, (train_idx, val_idx) in enumerate(skf.split(train_info.index, labels)):
    print(f'Fold {fold+1}')

    # 폴드별 train/validation 데이터 분리
    train_subset_df = train_info.iloc[train_idx]
    val_subset_df = train_info.iloc[val_idx]

    # CustomDataset 생성
    train_dataset = CustomDataset(
        root_dir=traindata_dir,
        info_df=train_subset_df,
        transform=train_transform
    )
    val_dataset = CustomDataset(
        root_dir=traindata_dir,
        info_df=val_subset_df,
        transform=val_transform
    )

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    # 모델 생성
    model_selector = ModelSelector(
        model_type='timm', 
        num_classes=num_classes,
        model_name='resnet101', 
        pretrained=True
    )
    model = model_selector.get_model()
    model.to(device)

    # Optimizer 및 Scheduler 설정
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
    loss_fn = Loss()

    # Trainer 설정 및 학습 진행
    trainer = Trainer(
        model=model,
        device=device,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        loss_fn=loss_fn,
        epochs=30,
        result_path=save_result_path,
        fold=fold + 1  
    )

    # 폴드별 학습 진행
    trainer.train()

    # 학습된 모델 저장
    models.append(model)


Fold 1
Epoch 1/30


Train Accuracy: 30.34%


Validation Accuracy: 63.14%
Epoch 1, Train Loss: 3.4853, Validation Loss: 1.1327

Save 0epoch result. Loss = 1.1327
Epoch 2/30


Train Accuracy: 71.75%


Validation Accuracy: 73.59%
Epoch 2, Train Loss: 0.7470, Validation Loss: 0.7656

Save 1epoch result. Loss = 0.7656
Epoch 3/30


...

Validation Accuracy: 82.62%
Epoch 28, Train Loss: 0.0128, Validation Loss: 0.5317

Save 27epoch result. Loss = 0.5317
Epoch 29/30


Train Accuracy: 99.03%


Validation Accuracy: 81.76%
Epoch 29, Train Loss: 0.0101, Validation Loss: 0.5289

Save 28epoch result. Loss = 0.5289
Epoch 30/30


Train Accuracy: 98.88%


Validation Accuracy: 82.36%
Epoch 30, Train Loss: 0.0113, Validation Loss: 0.5345



# Inference

In [20]:
# 앙상블 예측을 위한 함수
def ensemble_inference(
    models: List[nn.Module], 
    device: torch.device, 
    test_loader: DataLoader,
    num_classes: int
):
    # 모델들을 평가 모드로 설정
    for model in models:
        model.to(device)
        model.eval()

    # 예측값을 저장할 배열을 초기화
    predictions = np.zeros((len(test_loader.dataset), num_classes))  # (데이터셋 크기, 클래스 수)

    # 인덱스를 추적하기 위한 변수
    start_idx = 0

    with torch.no_grad():  # Gradient 계산 비활성화
        for images in tqdm(test_loader):
            batch_size = images.size(0)  # 현재 배치의 크기
            images = images.to(device)
            
            # 각 배치별 모델의 예측을 저장할 배열을 초기화
            fold_predictions = np.zeros((batch_size, num_classes))

            # 각 모델에 대해 예측 수행
            for model in models:
                logits = model(images)
                fold_predictions += logits.cpu().numpy()  # 예측값을 합산 (logits 사용)

            # 각 모델의 예측값을 합산 후 모델 수로 나눠 평균
            fold_predictions /= len(models)
            
            # 예측값을 전체 predictions 배열에 저장
            predictions[start_idx:start_idx + batch_size] = fold_predictions
            start_idx += batch_size

    # 최종 예측값은 확률값 중 가장 큰 클래스로 결정
    final_predictions = np.argmax(predictions, axis=1)

    return final_predictions


In [23]:
# 추론 데이터의 경로와 정보를 가진 파일의 경로를 설정.
testdata_dir = "../data/test"
testdata_info_file = "../data/test.csv"
save_result_path = "../train_result"

In [24]:
# 추론 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
test_info = pd.read_csv(testdata_info_file)

# 총 class 수.
num_classes = 500


In [25]:
# 추론에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
test_transform = transform_selector.get_transform(is_train=False)

# 추론에 사용할 Dataset을 선언.
test_dataset = CustomDataset(
    root_dir=testdata_dir,
    info_df=test_info,
    transform=test_transform,
    is_inference=True
)

# 추론에 사용할 DataLoader를 선언.
test_loader = DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=False,
    drop_last=False
)

In [28]:
# 추론에 사용할 장비를 선택.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 앙상블에 사용할 모델들을 불러오기.
model_paths = [
    os.path.join(save_result_path, "model_fold_1_epoch_12_loss_0.5175.pt"),
    os.path.join(save_result_path, "model_fold_2_epoch_27_loss_0.4591.pt"),
    os.path.join(save_result_path, "model_fold_3_epoch_19_loss_0.5521.pt"),
    os.path.join(save_result_path, "model_fold_4_epoch_12_loss_0.4968.pt"),
    os.path.join(save_result_path, "model_fold_5_epoch_23_loss_0.5138.pt"),
    os.path.join(save_result_path, "model_fold_6_epoch_10_loss_0.5614.pt"),
    os.path.join(save_result_path, "model_fold_7_epoch_16_loss_0.4750.pt"),
    os.path.join(save_result_path, "model_fold_8_epoch_21_loss_0.5544.pt"),
    os.path.join(save_result_path, "model_fold_9_epoch_28_loss_0.5352.pt"),
    os.path.join(save_result_path, "model_fold_10_epoch_28_loss_0.5289.pt"),
]


In [29]:
# 모델 리스트 초기화
models = []

for model_path in model_paths:
    model_selector = ModelSelector(
        model_type='timm', 
        num_classes=num_classes,
        model_name='resnet101', 
        pretrained=False
    )
    model = model_selector.get_model()
    model.load_state_dict(torch.load(model_path, map_location=device))
    models.append(model)

# 앙상블 추론 수행
predictions = ensemble_inference(
    models=models, 
    device=device, 
    test_loader=test_loader,
    num_classes=num_classes
)

100%|██████████| 157/157 [03:21<00:00,  1.28s/it]


In [30]:
# predictions를 CSV에 저장할 때 형식을 맞춰서 저장
test_info['target'] = predictions
test_info = test_info.reset_index().rename(columns={"index": "ID"})

In [31]:
# DataFrame 저장
test_info.to_csv("resnet101_fold10_sharpning_ensemble.csv", index=False)
